In [25]:
!pip install pandas numpy
!pip install matplotlib seaborn
!pip install pulp
!pip install highspy   
!pip install pulp[coin-or]   # ensures optimized CBC build
!pip install xgboost
!pip install joblib
!pip install scikit-learn
!pip install -U scikit-learn



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\ranjo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\ranjo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\ranjo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\ranjo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#'

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\ranjo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\ranjo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\ranjo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\ranjo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\ranjo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [26]:
# ==============================================
# Part 1: Load dataset & compute baseline 
# ==============================================

import pandas as pd
import numpy as np

# -------- 1) Load --------
file_path = "apartments_toronto_by_fsa-FINAL.csv"   # or your chosen filename
df = pd.read_csv(file_path)
print("Dataset shape:", df.shape)
display(df.head())

# -------- 2) Ensure numeric types on required columns --------
required_numeric = [
    "num_units", "Total_EV_Pop", "Population", "Average_Household_Size"
]

for col in required_numeric:
    if col not in df.columns:
        raise KeyError(f"Missing required column: {col}")

df[required_numeric] = df[required_numeric].apply(pd.to_numeric, errors="coerce")

# Report any rows that became NaN after coercion
nan_counts = df[required_numeric].isna().sum()
if nan_counts.any():
    print("\n⚠️ Non-numeric or missing values detected (will be dropped for calc):")
    print(nan_counts[nan_counts > 0])

# Keep a copy before dropping, if you want to inspect later
df_raw = df.copy()

# Drop rows that cannot be used for the calculation
df = df.dropna(subset=required_numeric).copy()
dropped = len(df_raw) - len(df)
if dropped > 0:
    print(f"\nℹ️ Dropped {dropped} row(s) with missing required numeric values.")

# -------- 3) Helper computations --------
# num_residents_calc = num_units * Average_Household_Size
df["num_residents_calc"] = df["num_units"] * df["Average_Household_Size"]

# ev_adoption_rate_calc = Total_EV_Pop / Population  (handle division-by-zero)
df["ev_adoption_rate_calc"] = np.where(
    df["Population"] == 0, 
    np.nan, 
    df["Total_EV_Pop"] / df["Population"]
)

# -------- 4) Final prediction (baseline math, not ML) --------
# predicted_calc = num_residents_calc * ev_adoption_rate_calc
df["Predicted_EV_Owners_Calc"] = df["num_residents_calc"] * df["ev_adoption_rate_calc"]

# -------- 5) Optional validation against existing columns --------
# If your CSV already includes 'num_residents' and/or 'Predicted EV Owners Per Building',
# compute small diffs to confirm the formula matches.
if "num_residents" in df.columns:
    df["num_residents_diff"] = df["num_residents_calc"] - pd.to_numeric(df["num_residents"], errors="coerce")

if "Predicted EV Owners Per Building" in df.columns:
    df["Predicted_diff"] = (
        df["Predicted_EV_Owners_Calc"] 
        - pd.to_numeric(df["Predicted EV Owners Per Building"], errors="coerce")
    )

# -------- 6) Preview nicely --------
preview_cols = [
    # identifiers (if present)
    *(c for c in ["FSA", "SITE_ADDRESS", "building_name", "postal_code"] if c in df.columns),
    "num_units", "Average_Household_Size", "Population", "Total_EV_Pop",
    "num_residents_calc", "ev_adoption_rate_calc", "Predicted_EV_Owners_Calc"
]

# Add diffs if they exist
for c in ["num_residents_diff", "Predicted_diff"]:
    if c in df.columns:
        preview_cols.append(c)

pd.set_option("display.float_format", lambda x: f"{x:.6f}")
print("\n✅ Baseline calculation complete. Preview:")
display(df[preview_cols].head(10))

# -------- 7) Save a clean output for downstream (ROI / ML later) --------
output_cols = [
    *(c for c in ["FSA", "SITE_ADDRESS", "building_name", "postal_code"] if c in df.columns),
    "num_units", "Average_Household_Size", "Population", "Total_EV_Pop",
    "num_residents_calc", "ev_adoption_rate_calc", "Predicted_EV_Owners_Calc"
]
df_out = df[output_cols].rename(columns={
    "num_residents_calc": "num_residents_baseline",
    "ev_adoption_rate_calc": "ev_adoption_rate_baseline",
    "Predicted_EV_Owners_Calc": "Predicted_EV_Owners_Baseline"
})
df_out.to_csv("baseline_predicted_ev_owners.csv", index=False)
print("\n💾 Saved: baseline_predicted_ev_owners.csv")


Dataset shape: (3598, 11)


,num_units,FSA,SITE_ADDRESS,PHEV_Count,BEV_Count,Total_EV_Pop,Population,Average_Household_Size,num_residents,ev_adoption_rate,Predicted EV Owners Per Building
0,35.000000,M6K,301 LANSDOWNE AVE,99.000000,289.000000,388.000000,42110.000000,1.852944,64.853032,0.009214,0.597553
1,12.000000,M6K,1505 DUNDAS ST W,99.000000,289.000000,388.000000,42110.000000,1.852944,22.235325,0.009214,0.204875
2,19.000000,M6H,156 BRANDON AVE,140.000000,232.000000,372.000000,43730.000000,2.270037,43.130710,0.008507,0.366902
3,15.000000,M6H,109 GLENHOLME AVE,140.000000,232.000000,372.000000,43730.000000,2.270037,34.050561,0.008507,0.289659
4,106.000000,M5V,679 QUEENS QUAY W,147.000000,700.000000,847.000000,59912.000000,1.612054,170.877761,0.014137,2.415768



⚠️ Non-numeric or missing values detected (will be dropped for calc):
num_units                 75
Total_EV_Pop              75
Population                75
Average_Household_Size    75
dtype: int64

ℹ️ Dropped 75 row(s) with missing required numeric values.

✅ Baseline calculation complete. Preview:


,FSA,SITE_ADDRESS,num_units,Average_Household_Size,Population,Total_EV_Pop,num_residents_calc,ev_adoption_rate_calc,Predicted_EV_Owners_Calc,num_residents_diff,Predicted_diff
0,M6K,301 LANSDOWNE AVE,35.000000,1.852944,42110.000000,388.000000,64.853032,0.009214,0.597553,-0.000000,0.000000
1,M6K,1505 DUNDAS ST W,12.000000,1.852944,42110.000000,388.000000,22.235325,0.009214,0.204875,0.000000,0.000000
2,M6H,156 BRANDON AVE,19.000000,2.270037,43730.000000,372.000000,43.130710,0.008507,0.366902,-0.000000,0.000000
3,M6H,109 GLENHOLME AVE,15.000000,2.270037,43730.000000,372.000000,34.050561,0.008507,0.289659,-0.000000,0.000000
4,M5V,679 QUEENS QUAY W,106.000000,1.612054,59912.000000,847.000000,170.877761,0.014137,2.415768,0.000000,-0.000000
5,M6J,15 GORE VALE AVE,10.000000,2.077978,32111.000000,385.000000,20.779784,0.011990,0.249143,0.000000,0.000000
6,M6J,14 RUSHOLME DR,22.000000,2.077978,32111.000000,385.000000,45.715524,0.011990,0.548114,0.000000,-0.000000
7,M6H,323 RUSHOLME RD,226.000000,2.270037,43730.000000,372.000000,513.028447,0.008507,4.364203,-0.000000,0.000000
8,M6H,1 HAVELOCK ST,18.000000,2.270037,43730.000000,372.000000,40.860673,0.008507,0.347591,0.000000,0.000000
9,M5R,34 WALMER RD,61.000000,1.892163,26197.000000,524.000000,115.421957,0.020002,2.308704,-0.000000,0.000000



💾 Saved: baseline_predicted_ev_owners.csv


In [27]:
import pandas as pd

df = pd.read_csv("baseline_predicted_ev_owners.csv")
print(df.columns.tolist())


['FSA', 'SITE_ADDRESS', 'num_units', 'Average_Household_Size', 'Population', 'Total_EV_Pop', 'num_residents_baseline', 'ev_adoption_rate_baseline', 'Predicted_EV_Owners_Baseline']


In [28]:
import pandas as pd

df = pd.read_csv("baseline_predicted_ev_owners.csv")
print("\n✅ Column names in your dataset:")
for col in df.columns:
    print(f" - {col}")



✅ Column names in your dataset:
 - FSA
 - SITE_ADDRESS
 - num_units
 - Average_Household_Size
 - Population
 - Total_EV_Pop
 - num_residents_baseline
 - ev_adoption_rate_baseline
 - Predicted_EV_Owners_Baseline


In [29]:
# ============================================================
# PART 2 — ENGINEERING-ACCURATE L2 CHARGER PLANNING MODEL
# ============================================================

import pandas as pd
import numpy as np
import math

# ------------------------------------------------------------
# CONFIG (all values pulled from your EV Variable Breakdown PDF)
# ------------------------------------------------------------

INPUT_FILE  = "baseline_predicted_ev_owners.csv"
OUTPUT_FILE = "engineering_charger_plan.csv"
EV_OWNER_COL = "Predicted_EV_Owners_Baseline"

CONFIG = {
    # Demand model (official)
    "sessions_per_week_per_ev": 4.0,       # PDF: EV charges 3–4 times weekly (choose 4 for safety)
    "hours_available_per_day": 16.0,       # PDF assumption
    "session_time_hours": 3.0,             # PDF: time to charge per session
    "sessions_per_charger_per_day": 5.33,  # PDF: 16 hours ÷ 3 hours = 5.33 sessions/day

    # Charger electrical specifications
    "charger_power_kw": 7.0,               # PDF: Level 2 charger (7 kW)
    "charging_efficiency": 0.90,           # PDF: 90% efficiency

    # Energy delivered per session
    "energy_per_session_kwh": 21.0,        # PDF: 7kW × 3hr = 21 kWh

    # Economics
    "price_to_user_per_kwh": 0.30,         # PDF
    "utility_energy_cost_per_kwh": 0.20,   # PDF
    "demand_charge_per_kw_month": 14.0,    # PDF: $14/kW/month
    "maintenance_per_charger_per_month": 33.0,  # PDF

    # Capex
    "setup_cost_per_charger": 8000.0,      # PDF

    # Min requirement
    "min_chargers": 1
}

# ------------------------------------------------------------
# ENGINEERING FUNCTIONS
# ------------------------------------------------------------

def daily_sessions_from_owners(ev_owners, cfg):
    return (ev_owners * cfg["sessions_per_week_per_ev"]) / 7.0

def chargers_needed_engineering(ev_owners, cfg):
    daily_sessions = daily_sessions_from_owners(ev_owners, cfg)
    cap = cfg["sessions_per_charger_per_day"]

    if daily_sessions <= 0:
        return 0

    needed = math.ceil(daily_sessions / cap)
    return max(needed, cfg["min_chargers"])

def daily_revenue(daily_sessions, cfg):
    return daily_sessions * cfg["energy_per_session_kwh"] * cfg["price_to_user_per_kwh"]

def daily_energy_cost(daily_sessions, cfg):
    grid_kwh = daily_sessions * (cfg["energy_per_session_kwh"] / cfg["charging_efficiency"])
    return grid_kwh * cfg["utility_energy_cost_per_kwh"]

def daily_demand_charge(n_chargers, cfg):
    monthly = cfg["demand_charge_per_kw_month"] * cfg["charger_power_kw"] * n_chargers
    return monthly / 30.0

def daily_maintenance(n_chargers, cfg):
    return (cfg["maintenance_per_charger_per_month"] * n_chargers) / 30.0

def break_even_days(total_capex, profit):
    if profit <= 0:
        return np.inf
    return total_capex / profit

def compute_row(row, cfg, col):
    owners = float(row[col])

    daily_sessions = daily_sessions_from_owners(owners, cfg)
    n_ch = chargers_needed_engineering(owners, cfg)

    revenue = daily_revenue(daily_sessions, cfg)
    e_cost = daily_energy_cost(daily_sessions, cfg)
    d_cost = daily_demand_charge(n_ch, cfg)
    m_cost = daily_maintenance(n_ch, cfg)

    daily_profit = revenue - e_cost - d_cost - m_cost
    capex = cfg["setup_cost_per_charger"] * n_ch
    be = break_even_days(capex, daily_profit)

    return pd.Series({
        "Predicted_EV_Owners": owners,
        "Daily_Sessions": daily_sessions,
        "Chargers_Needed": n_ch,
        "Setup_Cost_CAD": capex,
        "Daily_Revenue_CAD": revenue,
        "Daily_Energy_Cost_CAD": e_cost,
        "Daily_Demand_Charge_CAD": d_cost,
        "Daily_Maintenance_CAD": m_cost,
        "Daily_Profit_CAD": daily_profit,
        "BreakEven_Days": be
    })

# ------------------------------------------------------------
# LOAD, EXECUTE, SAVE
# ------------------------------------------------------------

df = pd.read_csv(INPUT_FILE)

if EV_OWNER_COL not in df.columns:
    raise ValueError(f"Missing EV owner column: {EV_OWNER_COL}")

df[EV_OWNER_COL] = pd.to_numeric(df[EV_OWNER_COL], errors="coerce").fillna(0)

calc = df.apply(lambda r: compute_row(r, CONFIG, EV_OWNER_COL), axis=1)
out = pd.concat([df, calc], axis=1)

display(out.head(20))

out.to_csv(OUTPUT_FILE, index=False)
print(f"\n💾 Engineering charger planning saved → {OUTPUT_FILE}")


,FSA,SITE_ADDRESS,num_units,Average_Household_Size,Population,Total_EV_Pop,num_residents_baseline,ev_adoption_rate_baseline,Predicted_EV_Owners_Baseline,Predicted_EV_Owners,Daily_Sessions,Chargers_Needed,Setup_Cost_CAD,Daily_Revenue_CAD,Daily_Energy_Cost_CAD,Daily_Demand_Charge_CAD,Daily_Maintenance_CAD,Daily_Profit_CAD,BreakEven_Days
0,M6K,301 LANSDOWNE AVE,35.000000,1.852944,42110.000000,388.000000,64.853032,0.009214,0.597553,0.597553,0.341459,1.000000,8000.000000,2.151192,1.593476,3.266667,1.100000,-3.808950,inf
1,M6K,1505 DUNDAS ST W,12.000000,1.852944,42110.000000,388.000000,22.235325,0.009214,0.204875,0.204875,0.117072,1.000000,8000.000000,0.737552,0.546335,3.266667,1.100000,-4.175450,inf
2,M6H,156 BRANDON AVE,19.000000,2.270037,43730.000000,372.000000,43.130710,0.008507,0.366902,0.366902,0.209658,1.000000,8000.000000,1.320847,0.978405,3.266667,1.100000,-4.024225,inf
3,M6H,109 GLENHOLME AVE,15.000000,2.270037,43730.000000,372.000000,34.050561,0.008507,0.289659,0.289659,0.165520,1.000000,8000.000000,1.042774,0.772425,3.266667,1.100000,-4.096318,inf
4,M5V,679 QUEENS QUAY W,106.000000,1.612054,59912.000000,847.000000,170.877761,0.014137,2.415768,2.415768,1.380439,1.000000,8000.000000,8.696763,6.442047,3.266667,1.100000,-2.111950,inf
5,M6J,15 GORE VALE AVE,10.000000,2.077978,32111.000000,385.000000,20.779784,0.011990,0.249143,0.249143,0.142367,1.000000,8000.000000,0.896913,0.664380,3.266667,1.100000,-4.134134,inf
6,M6J,14 RUSHOLME DR,22.000000,2.077978,32111.000000,385.000000,45.715524,0.011990,0.548114,0.548114,0.313208,1.000000,8000.000000,1.973209,1.461636,3.266667,1.100000,-3.855094,inf
7,M6H,323 RUSHOLME RD,226.000000,2.270037,43730.000000,372.000000,513.028447,0.008507,4.364203,4.364203,2.493830,1.000000,8000.000000,15.711130,11.637874,3.266667,1.100000,-0.293411,inf
8,M6H,1 HAVELOCK ST,18.000000,2.270037,43730.000000,372.000000,40.860673,0.008507,0.347591,0.347591,0.198624,1.000000,8000.000000,1.251329,0.926910,3.266667,1.100000,-4.042248,inf
9,M5R,34 WALMER RD,61.000000,1.892163,26197.000000,524.000000,115.421957,0.020002,2.308704,2.308704,1.319259,1.000000,8000.000000,8.311333,6.156543,3.266667,1.100000,-2.211877,inf



💾 Engineering charger planning saved → engineering_charger_plan.csv


In [30]:
# ==============================================================
# PART 4 — Strong AI Model for Predicting Optimal Chargers
# ==============================================================

import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
warnings.filterwarnings("ignore")

# ------------------------------------------------------------
# LOAD ENGINEERING OUTPUT FROM PART 2
# ------------------------------------------------------------
INPUT = "engineering_charger_plan.csv"
df = pd.read_csv(INPUT)

print(f"📂 Loaded engineering dataset: {len(df):,} rows")

target = "Chargers_Needed"

base_features = [
    "num_units",
    "Average_Household_Size",
    "Population",
    "Total_EV_Pop",
    "num_residents_baseline",
    "ev_adoption_rate_baseline",
    "Predicted_EV_Owners_Baseline"
]

# ------------------------------------------------------------
# FEATURE ENGINEERING (STRONGER)
# ------------------------------------------------------------
df["EV_penetration_rate"] = df["Predicted_EV_Owners_Baseline"] / df["num_units"]
df["EV_density_per_100_units"] = df["Predicted_EV_Owners_Baseline"] / (df["num_units"] / 100)
df["Residents_per_EV"] = df["num_residents_baseline"] / (df["Predicted_EV_Owners_Baseline"] + 1)

features = base_features + [
    "EV_penetration_rate",
    "EV_density_per_100_units",
    "Residents_per_EV"
]

X = df[features]
y = df[target]

# ------------------------------------------------------------
# SPLIT DATA
# ------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=42
)

print(f"✓ Train={len(X_train):,} | Test={len(X_test):,}")

# ------------------------------------------------------------
# TRAIN + EVALUATE MODELS
# ------------------------------------------------------------
def fit_model(name, model):
    t0 = time.time()
    pipe = Pipeline([("scaler", StandardScaler()), ("model", model)])
    pipe.fit(X_train, y_train)
    preds = np.clip(pipe.predict(X_test), 0, None)

    r2  = r2_score(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    mae  = mean_absolute_error(y_test, preds)

    print(f"  🔧 {name:<12} | R²={r2:.4f}  RMSE={rmse:.3f}  MAE={mae:.3f}  ({time.time()-t0:.2f}s)")
    return pipe, preds, r2

print("\n🔨 Training AI models...")

svr_pipe, svr_preds, svr_r2 = fit_model("SVR (RBF)", SVR(C=20, epsilon=0.1, gamma="scale"))
rf_pipe,  rf_preds,  rf_r2  = fit_model("RandomForest", RandomForestRegressor(n_estimators=400, n_jobs=-1, random_state=42))
xgb_pipe, xgb_preds, xgb_r2 = fit_model("XGBoost", XGBRegressor(n_estimators=500, max_depth=6, learning_rate=0.05, n_jobs=-1, random_state=42, verbosity=0))

# ------------------------------------------------------------
# ENSEMBLE (WEIGHTED BY R²)
# ------------------------------------------------------------
weights = np.array([svr_r2, rf_r2, xgb_r2])
weights = weights / weights.sum()

ensemble_float = (
    svr_preds * weights[0] +
    rf_preds  * weights[1] +
    xgb_preds * weights[2]
)

ensemble = np.clip(np.round(ensemble_float), 1, None).astype(int)

ens_r2 = r2_score(y_test, ensemble)
ens_rmse = np.sqrt(mean_squared_error(y_test, ensemble))
ens_mae = mean_absolute_error(y_test, ensemble)

print(f"\n⭐ ENSEMBLE | R²={ens_r2:.4f}  RMSE={ens_rmse:.3f}  MAE={ens_mae:.3f}")

# ------------------------------------------------------------
# CONFIDENCE INTERVALS
# ------------------------------------------------------------
print("\n📈 Computing 95% confidence intervals...")

n_boot = 200
boot = np.zeros((n_boot, len(ensemble_float)))
rng = np.random.default_rng(42)

for i in range(n_boot):
    idx = rng.choice(len(ensemble_float), len(ensemble_float), replace=True)
    boot[i] = ensemble_float[idx]

low = np.round(np.percentile(boot, 2.5, axis=0)).astype(int)
high = np.round(np.percentile(boot, 97.5, axis=0)).astype(int)

# ------------------------------------------------------------
# DETAILED OUTPUT
# ------------------------------------------------------------
df_results = pd.DataFrame({
    "FSA": df.loc[y_test.index, "FSA"],
    "SITE_ADDRESS": df.loc[y_test.index, "SITE_ADDRESS"],
    "Actual_Chargers": y_test,
    "Predicted_Chargers": ensemble,
    "Lower_95CI": low,
    "Upper_95CI": high,
    "Absolute_Error": abs(y_test - ensemble)
})

df_results.to_csv("ai_part4_predictions.csv", index=False)
display(df_results.head(20))

print("\n💾 Saved → ai_part4_predictions.csv")


📂 Loaded engineering dataset: 3,523 rows
✓ Train=1,761 | Test=1,762

🔨 Training AI models...
  🔧 SVR (RBF)    | R²=0.8325  RMSE=0.125  MAE=0.072  (0.05s)
  🔧 RandomForest | R²=0.9851  RMSE=0.037  MAE=0.003  (0.25s)
  🔧 XGBoost      | R²=0.9393  RMSE=0.075  MAE=0.006  (0.10s)

⭐ ENSEMBLE | R²=0.9395  RMSE=0.075  MAE=0.006

📈 Computing 95% confidence intervals...


,FSA,SITE_ADDRESS,Actual_Chargers,Predicted_Chargers,Lower_95CI,Upper_95CI,Absolute_Error
1637,M8X,2688 BLOOR ST W,1.000000,1,1,2,0.000000
1656,M9W,9 BERGAMOT AVE,1.000000,1,1,2,0.000000
2856,M1K,2356 EGLINTON AVE E,1.000000,1,1,2,0.000000
3117,M6G,469 PALMERSTON BLVD,1.000000,1,1,2,0.000000
3070,M4T,77 ST CLAIR AVE E,2.000000,2,1,2,0.000000
2743,M6R,1609 QUEEN ST W,1.000000,1,1,2,0.000000
144,M1K,10 GORDONRIDGE PL,1.000000,1,1,2,0.000000
299,M1P,4 TREEWOOD ST,1.000000,1,1,2,0.000000
1105,M6L,338 FALSTAFF AVE,1.000000,1,1,2,0.000000
2186,M9P,17 RIVERVIEW HTS,1.000000,1,1,2,0.000000



💾 Saved → ai_part4_predictions.csv


In [ ]:
# ==============================================================
# PART 5 — ROI FORECASTING FOR EV CHARGER PROJECTS 
# ==============================================================

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Your input (from Part 2)
INPUT_FILE = "engineering_charger_plan.csv"
OUTPUT_FILE = "roi_full_forecast.csv"

df = pd.read_csv(INPUT_FILE)
print(f"📂 Loaded Part 2 engineering file: {len(df)} buildings")

# ------------------------------------------------------------
# Detect correct CAPEX column automatically
# ------------------------------------------------------------

capex_col_candidates = [
    "Setup_Cost_per_Building_CAD",
    "Setup_Cost_per_Building",
    "Setup_Cost_CAD",
    "Setup_Cost"
]

capex_col = None
for c in capex_col_candidates:
    if c in df.columns:
        capex_col = c
        break

if capex_col is None:
    raise KeyError(
        f"❌ No setup cost column found. Expected one of: {capex_col_candidates}\n"
        f"Columns found: {df.columns.tolist()}"
    )

print(f"✔ Using CAPEX column: {capex_col}")

# ------------------------------------------------------------
# ROI CONFIG
# ------------------------------------------------------------

ROI = {
    "days_per_month":           30.0,
    "months_per_year":          12.0,
    "annual_discount_rate":     0.06,
    "project_lifetime_years":   10
}

# ------------------------------------------------------------
# Helper function
# ------------------------------------------------------------

def compute_roi_row(row):
    daily_profit = float(row["Daily_Profit_CAD"])
    capex = float(row[capex_col])

    monthly_profit = daily_profit * ROI["days_per_month"]
    yearly_profit = monthly_profit * ROI["months_per_year"]

    # ROI calculations
    roi_1 = (yearly_profit - capex) / capex
    roi_3 = ((yearly_profit * 3) - capex) / capex
    roi_5 = ((yearly_profit * 5) - capex) / capex
    roi_10 = ((yearly_profit * 10) - capex) / capex

    # Break-even
    if daily_profit <= 0:
        be_days = np.inf
        be_date = "NEVER"
    else:
        be_days = capex / daily_profit

        # Protect from datetime overflow
        try:
            be_date = (datetime.today() + timedelta(days=float(be_days))).strftime("%Y-%m-%d")
        except OverflowError:
            be_date = "DATE_TOO_LARGE"

    return pd.Series({
        "Monthly_Profit_CAD": monthly_profit,
        "Yearly_Profit_CAD": yearly_profit,
        "ROI_1yr": roi_1,
        "ROI_3yr": roi_3,
        "ROI_5yr": roi_5,
        "ROI_10yr": roi_10,
        "BreakEven_Days": be_days,
        "BreakEven_Date": be_date
    })

# ------------------------------------------------------------
# Apply function to each building
# ------------------------------------------------------------

roi_cols = df.apply(compute_roi_row, axis=1)
out = pd.concat([df, roi_cols], axis=1)

print("\n=== ROI Forecast Preview ===")
display(out.head(20))

# Save to CSV
out.to_csv(OUTPUT_FILE, index=False)
print(f"\n💾 ROI forecast saved → {OUTPUT_FILE}")


📂 Loaded Part 2 engineering file: 3523 buildings
✔ Using CAPEX column: Setup_Cost_CAD

=== ROI Forecast Preview ===


,FSA,SITE_ADDRESS,num_units,Average_Household_Size,Population,Total_EV_Pop,num_residents_baseline,ev_adoption_rate_baseline,Predicted_EV_Owners_Baseline,Predicted_EV_Owners,Daily_Sessions,Chargers_Needed,Setup_Cost_CAD,Daily_Revenue_CAD,Daily_Energy_Cost_CAD,Daily_Demand_Charge_CAD,Daily_Maintenance_CAD,Daily_Profit_CAD,BreakEven_Days,Monthly_Profit_CAD,Yearly_Profit_CAD,ROI_1yr,ROI_3yr,ROI_5yr,ROI_10yr,BreakEven_Days,BreakEven_Date
0,M6K,301 LANSDOWNE AVE,35.000000,1.852944,42110.000000,388.000000,64.853032,0.009214,0.597553,0.597553,0.341459,1.000000,8000.000000,2.151192,1.593476,3.266667,1.100000,-3.808950,inf,-114.268503,-1371.222036,-1.171403,-1.514208,-1.857014,-2.714028,inf,NEVER
1,M6K,1505 DUNDAS ST W,12.000000,1.852944,42110.000000,388.000000,22.235325,0.009214,0.204875,0.204875,0.117072,1.000000,8000.000000,0.737552,0.546335,3.266667,1.100000,-4.175450,inf,-125.263487,-1503.161841,-1.187895,-1.563686,-1.939476,-2.878952,inf,NEVER
2,M6H,156 BRANDON AVE,19.000000,2.270037,43730.000000,372.000000,43.130710,0.008507,0.366902,0.366902,0.209658,1.000000,8000.000000,1.320847,0.978405,3.266667,1.100000,-4.024225,inf,-120.726744,-1448.720930,-1.181090,-1.543270,-1.905451,-2.810901,inf,NEVER
3,M6H,109 GLENHOLME AVE,15.000000,2.270037,43730.000000,372.000000,34.050561,0.008507,0.289659,0.289659,0.165520,1.000000,8000.000000,1.042774,0.772425,3.266667,1.100000,-4.096318,inf,-122.889535,-1474.674419,-1.184334,-1.553003,-1.921672,-2.843343,inf,NEVER
4,M5V,679 QUEENS QUAY W,106.000000,1.612054,59912.000000,847.000000,170.877761,0.014137,2.415768,2.415768,1.380439,1.000000,8000.000000,8.696763,6.442047,3.266667,1.100000,-2.111950,inf,-63.358509,-760.302112,-1.095038,-1.285113,-1.475189,-1.950378,inf,NEVER
5,M6J,15 GORE VALE AVE,10.000000,2.077978,32111.000000,385.000000,20.779784,0.011990,0.249143,0.249143,0.142367,1.000000,8000.000000,0.896913,0.664380,3.266667,1.100000,-4.134134,inf,-124.024008,-1488.288099,-1.186036,-1.558108,-1.930180,-2.860360,inf,NEVER
6,M6J,14 RUSHOLME DR,22.000000,2.077978,32111.000000,385.000000,45.715524,0.011990,0.548114,0.548114,0.313208,1.000000,8000.000000,1.973209,1.461636,3.266667,1.100000,-3.855094,inf,-115.652818,-1387.833819,-1.173479,-1.520438,-1.867396,-2.734792,inf,NEVER
7,M6H,323 RUSHOLME RD,226.000000,2.270037,43730.000000,372.000000,513.028447,0.008507,4.364203,4.364203,2.493830,1.000000,8000.000000,15.711130,11.637874,3.266667,1.100000,-0.293411,inf,-8.802326,-105.627907,-1.013203,-1.039610,-1.066017,-1.132035,inf,NEVER
8,M6H,1 HAVELOCK ST,18.000000,2.270037,43730.000000,372.000000,40.860673,0.008507,0.347591,0.347591,0.198624,1.000000,8000.000000,1.251329,0.926910,3.266667,1.100000,-4.042248,inf,-121.267442,-1455.209302,-1.181901,-1.545703,-1.909506,-2.819012,inf,NEVER
9,M5R,34 WALMER RD,61.000000,1.892163,26197.000000,524.000000,115.421957,0.020002,2.308704,2.308704,1.319259,1.000000,8000.000000,8.311333,6.156543,3.266667,1.100000,-2.211877,inf,-66.356302,-796.275623,-1.099534,-1.298603,-1.497672,-1.995345,inf,NEVER



💾 ROI forecast saved → roi_full_forecast.csv
